<a href="https://colab.research.google.com/github/chomamat/fit-bp/blob/master/13_niklaus_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp gdrive/My\ Drive/x64/*.npy .
!cp gdrive/My\ Drive/*.py .
!ls -l

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import _torch_tools as tt
import _my_tools as mt
import numpy as np
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is", device)
X_train, y_train, X_test, y_test = mt.loadData("dataset_interpolation/",'float16',channels_last=False)

In [ ]:
FILTER_SIZE = 31
NUM_WORKERS = 10

# taken from Kartasev et al.
class SeparableConvolutionSlow(nn.Module):
    def __init__(self):
        super(SeparableConvolutionSlow, self).__init__()
    
    def forward(self, im, vertical, horizontal):
        n_b = im.size(0)
        n_channels = im.size(1)
        m = im.size(2)
        m_out = m - FILTER_SIZE + 1
        
        assert im.size(2) == im.size(3)
        assert vertical.size(0) == horizontal.size(0) == n_b
        assert vertical.size(1) == horizontal.size(1) == FILTER_SIZE
        assert vertical.size(2) == horizontal.size(2) == vertical.size(3) == horizontal.size(3) == m_out

        n_workers = NUM_WORKERS

        if os.name == 'nt' and NUM_WORKERS > 1:
#             print('Parallel Separable Convolution on CPU not supported on Windows. Proceeding on main thread...')
            n_workers = 1

        if vertical.requires_grad and NUM_WORKERS > 1:
#             print('Parallel Separable Convolution on CPU not supported during training. Proceeding on main thread...')
            n_workers = 1

        output = im.new().resize_(n_b, n_channels, m_out, m_out).zero_()

        if n_workers > 1:
            return parallel_sep_conv(im, horizontal, vertical, output, n_workers)
        else:
            return sep_conv(im, horizontal, vertical, output)


def local_separable_conv_2d(im, horizontal, vertical, output=None):
    """im: [n_channels x m x m], horizontal: [51 x m x m], vertical: [51 x m x m]
       -> return: [n_channels x (m - 50) x (m - 50)]"""
    n_channels = im.size(0)
    m = im.size(1)
    m_out = m - FILTER_SIZE + 1
    if output is None:
        output = t.zeros((n_channels, m_out, m_out))
    for row in range(m_out):
        for col in range(m_out):
            sub_patch = im[:, row:row + FILTER_SIZE, col:col + FILTER_SIZE]
            local_horiz = horizontal[:, row, col]
            local_vert = vertical[:, row, col].view(-1, 1)
            output[:, row, col] = (sub_patch * local_horiz * local_vert).sum(dim=1).sum(dim=1)
    return output


def _sep_conv_worker(im, horizontal, vertical, output, worker_batch_size, offset):
    n_b = im.size(0)
    max_range = min(n_b, worker_batch_size+offset)
    for b in range(offset, max_range):
        local_separable_conv_2d(im[b], horizontal[b], vertical[b], output=output[b])
    return output


def sep_conv(im, horizontal, vertical, output):
    """
    Runs the separable convolution on multiple images sequentially on a single thread
    :param im: Input images as a tensor. im[0] must correspond to the first image of the batch
    :param horizontal: Set of horizontal filters as a tensor
    :param vertical: Set of vertical filters as a tensor
    :param output: Tensor used as output. Same shape as im. Must be passed pre-allocated and initialized with zeros
    :return: Tensor resulting from the convolution
    """
    return _sep_conv_worker(im, horizontal, vertical, output, im.size(0), 0)


def parallel_sep_conv(im, horizontal, vertical, output, n_workers):
    """
    Spawns the specified amount of workers to run the separable convolution on multiple images in parallel
    :param im: Input images as a tensor. im[0] must correspond to the first image of the batch
    :param horizontal: Set of horizontal filters as a tensor
    :param vertical: Set of vertical filters as a tensor
    :param output: Tensor used as output. Same shape as im. Must be passed pre-allocated and initialized with zeros
    :param n_workers: Number of workers to be used. Must be greater than zero
    :return: Tensor resulting from the convolution
    """

    n_b = im.size(0)
    n_workers = min(n_b, n_workers)
    worker_batch_size = n_b // n_workers
    processes = []

    output.share_memory_()

    for i in range(n_workers):

        offset = worker_batch_size * i
        if i == n_workers-1:
            worker_batch_size += n_b % n_workers

        p = mp.Process(target=_sep_conv_worker, args=(im, horizontal, vertical, output, worker_batch_size, offset,))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    return output

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.KERNEL_SIZE = 31
        self.activation = nn.PReLU()
        
        self.conv_setup = {
            'kernel' : (3,3),
            'stride' : (1,1),
            'padding' : 1,
            'activation' : self.activation
        }
        self.pooling_setup = {
            'kernel_size' : (2,2),
            'stride' : (2,2)
        }
        self.upsample_setup = {
            'scale_factor' : 2,
            'mode' : 'bilinear',
            'align_corners' : True
        }

        self.pooling_layer = nn.AvgPool2d(**self.pooling_setup)
        self.upsample_layer = nn.Upsample(**self.upsample_setup)
        self.pad = nn.ReplicationPad2d(self.KERNEL_SIZE // 2)
        
        self.conv32 = self._convBlock(2, 32, **self.conv_setup)
        self.conv64 = self._convBlock(32, 64, **self.conv_setup)
        self.conv128 = self._convBlock(64, 128, **self.conv_setup)
        self.conv256 = self._convBlock(128, 256, **self.conv_setup)
        
        self.conv256_256 = self._convBlock(256, 256, **self.conv_setup)

        self.upsample256 = self._upsampleBlock(self.upsample_layer, 256, 256, **self.conv_setup)
        self.deconv128 = self._convBlock(256, 128, **self.conv_setup)
        self.upsample128 = self._upsampleBlock(self.upsample_layer, 128, 128, **self.conv_setup)
        self.deconv64 = self._convBlock(128, 64, **self.conv_setup)
        self.upsample64 = self._upsampleBlock(self.upsample_layer, 64, 64, **self.conv_setup)
        
        self.kernel_1h = self._kernelBlock(self.upsample_layer, 64, self.KERNEL_SIZE, **self.conv_setup)
        self.kernel_1v = self._kernelBlock(self.upsample_layer, 64, self.KERNEL_SIZE, **self.conv_setup)
        self.kernel_2h = self._kernelBlock(self.upsample_layer, 64, self.KERNEL_SIZE, **self.conv_setup)
        self.kernel_2v = self._kernelBlock(self.upsample_layer, 64, self.KERNEL_SIZE, **self.conv_setup)
        
        self.separable_conv = SeparableConvolutionSlow()
        
    def forward(self, x):
        img1 = x[:,:1,:,:]
        img2 = x[:,1:,:,:]
        
        # compression
        
        x32 = self.conv32(x)
        x32_p = self.pooling_layer(x32)
        x64 = self.conv64(x32_p)
        x64_p = self.pooling_layer(x64)
        x128 = self.conv128(x64_p)
        x128_p = self.pooling_layer(x128)
        x256 = self.conv256(x128_p)
        x256_p = self.pooling_layer(x256)

        x = self.conv256_256(x256_p)

        # expansion

        x = self.upsample256(x)
        x += x256
        x = self.deconv128(x)

        x = self.upsample128(x)
        x += x128
        x = self.deconv64(x)

        x = self.upsample64(x)
        x += x64
        
        # creating kernels & image padding
        
        k_1h = self.kernel_1h(x)
        k_1v = self.kernel_1v(x)
        k_2h = self.kernel_2h(x)
        k_2v = self.kernel_2v(x)
        
        p_img1 = self.pad(img1)
        p_img2 = self.pad(img2)
                
        return self.separable_conv(p_img1, k_1v, k_1h) + self.separable_conv(p_img2, k_2v, k_2h)
    
    @staticmethod
    def _convBlock(in_channels, out_channels, kernel, stride, padding, activation):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, out_channels, kernel, stride, padding), activation
        )
    @staticmethod
    def _kernelBlock(upsample, in_channels, out_channels, kernel, stride, padding, activation):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, out_channels, kernel, stride, padding), activation,
            upsample,
            nn.Conv2d(out_channels, out_channels, kernel, stride, padding)
        )
    @staticmethod
    def _upsampleBlock(upsample, in_channels, out_channels, kernel, stride, padding, activation):
        return nn.Sequential(
            upsample,
            nn.Conv2d(in_channels, out_channels, kernel, stride, padding), activation
        )

In [ ]:
model = Model()
training = tt.Training(model, device, X_train, y_train, X_test, y_test)
training.fit(16, 2, val=True)

In [ ]:
out = model( torch.tensor(X_test[0:10], dtype=torch.float).to(device) )
for i in range(10):
    mt.compare(i, X_test, y_test, out.cpu().detach().numpy(), channels_last=False)

In [ ]:
torch.save(model.state_dict(), "weights")
!zip fig.zip *.png